# Builds table with exposure data for a directory tree

In [1]:
%pylab notebook
%matplotlib notebook

import os, glob

import numpy as np
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

import exifread
import rawpy
from astropy.table import Table

from datapath import DATA

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
# Extracts exposure EXIF data from a list of images

def get_metadata(dirpath):
    
    list_files = glob.glob(dirpath + '/*.ARW')
    
    table = Table(names=('ISO', 'Exptime', 'f/stop', 'Exposure', 'Date', 'File'), 
                  dtype=('f4','f4','f4','f4','S2', 'S2'))
    
    for filename in list_files:
        f = open(filename, 'rb')
        tags = exifread.process_file(f)
        iso = ''
        exptime = ''

        for tag in tags.keys():
            if tag in ['EXIF ExposureTime']:
                exptime = tags[tag]
            if tag in ['EXIF ISOSpeedRatings']:
                iso = tags[tag]
            if tag in ['Image DateTime']:
                date = tags[tag]
                
        index = filename.split('_')[-1].rfind('.')
        iso = float(str(iso))
        date = str(date)
        try:
            fstop = float(filename.split('_')[-1][:index][1:])
        except ValueError:
            fstop = 2.8
            
        exptime_str = str(exptime)
        if '/' in exptime_str:
            exptime_tokens = exptime_str.split('/')
            exptime = float(exptime_tokens[0]) / float(exptime_tokens[1])
        else:
            exptime = float(exptime_str)
        
        exposure = (exptime/3.2) * (iso/100) / (fstop/2.8)**2
        
        filename_stripped = filename.split('astrophotography_data')[1]
        
        table.add_row((iso, exptime, fstop, exposure, date, filename_stripped))

    table.sort('Date')    
    
    return table

In [3]:
basepath = os.path.join(DATA,'astrophotography_data/cls/')
dirpath = os.path.join(basepath,'with_filter')
t = get_metadata(dirpath)
t.pprint_all()

 ISO   Exptime f/stop Exposure         Date                      File             
------ ------- ------ -------- ------------------- -------------------------------
 200.0    30.0    2.8    18.75 2020:05:10 21:59:12  /cls/with_filter/I200_f2.8.ARW
 400.0    30.0    2.8     37.5 2020:05:10 21:59:57  /cls/with_filter/I400_f2.8.ARW
 800.0    30.0    2.8     75.0 2020:05:10 22:00:41  /cls/with_filter/I800_f2.8.ARW
1600.0    30.0    2.8    150.0 2020:05:10 22:01:29 /cls/with_filter/I1600_f2.8.ARW
 200.0    30.0    2.0    36.75 2020:05:10 22:02:53    /cls/with_filter/I200_f2.ARW
 400.0    30.0    2.0     73.5 2020:05:10 22:03:35    /cls/with_filter/I400_f2.ARW
 800.0    30.0    2.0    147.0 2020:05:10 22:04:20    /cls/with_filter/I800_f2.ARW
1600.0    30.0    2.0    294.0 2020:05:10 22:05:06   /cls/with_filter/I1600_f2.ARW
 200.0    30.0    4.0   9.1875 2020:05:10 22:06:19    /cls/with_filter/I200_f4.ARW
 400.0    30.0    4.0   18.375 2020:05:10 22:07:02    /cls/with_filter/I400_f4.ARW
 800

In [4]:
basepath = os.path.join(DATA,'astrophotography_data/cls/')
dirpath = os.path.join(basepath,'without_filter')
t = get_metadata(dirpath)
t.pprint_all()

 ISO   Exptime f/stop Exposure         Date                       File               
------ ------- ------ -------- ------------------- ----------------------------------
 200.0    30.0    2.8    18.75 2020:05:10 22:17:47  /cls/without_filter/I200_f2.8.ARW
 400.0    30.0    2.8     37.5 2020:05:10 22:18:30  /cls/without_filter/I400_f2.8.ARW
1600.0    30.0    2.8    150.0 2020:05:10 22:19:13 /cls/without_filter/I1600_f2.8.ARW
 200.0    30.0    4.0   9.1875 2020:05:10 22:21:08    /cls/without_filter/I200_f4.ARW
 400.0    30.0    4.0   18.375 2020:05:10 22:21:51    /cls/without_filter/I400_f4.ARW
 800.0    30.0    4.0    36.75 2020:05:10 22:22:33    /cls/without_filter/I800_f4.ARW
1600.0    30.0    4.0     73.5 2020:05:10 22:23:17   /cls/without_filter/I1600_f4.ARW


In [5]:
basepath = os.path.join(DATA,'astrophotography_data/')
dirpath = os.path.join(basepath,'assateague_2021')
t = get_metadata(dirpath)
t.pprint_all()

 ISO   Exptime f/stop Exposure         Date                     File            
------ ------- ------ -------- ------------------- -----------------------------
6400.0     3.2    2.8     64.0 2021:09:25 21:10:05 /assateague_2021/DSC08226.ARW
6400.0     3.2    2.8     64.0 2021:09:25 21:10:10 /assateague_2021/DSC08227.ARW
6400.0     3.2    2.8     64.0 2021:09:25 21:10:15 /assateague_2021/DSC08228.ARW
6400.0     3.2    2.8     64.0 2021:09:25 21:10:20 /assateague_2021/DSC08229.ARW
6400.0     3.2    2.8     64.0 2021:09:25 21:10:25 /assateague_2021/DSC08230.ARW
6400.0     3.2    2.8     64.0 2021:09:25 21:10:30 /assateague_2021/DSC08231.ARW
6400.0     3.2    2.8     64.0 2021:09:25 21:10:35 /assateague_2021/DSC08232.ARW
6400.0     3.2    2.8     64.0 2021:09:25 21:10:40 /assateague_2021/DSC08233.ARW
6400.0     3.2    2.8     64.0 2021:09:25 21:10:45 /assateague_2021/DSC08234.ARW
6400.0     3.2    2.8     64.0 2021:09:25 21:12:46 /assateague_2021/DSC08235.ARW
6400.0     3.2    2.8     64

In [6]:
basepath = os.path.join(DATA,'astrophotography_data/assateague/milky_way/')
dirpath = os.path.join(basepath,'raw')
t = get_metadata(dirpath)
t.pprint_all()

  ISO   Exptime f/stop Exposure         Date                         File                 
------- ------- ------ -------- ------------------- --------------------------------------
 1600.0    20.0    2.8    100.0 2019:07:29 21:33:07 /assateague/milky_way/raw/DSC03264.ARW
 3200.0    20.0    2.8    200.0 2019:07:29 21:34:45 /assateague/milky_way/raw/DSC03265.ARW
 6400.0    20.0    2.8    400.0 2019:07:29 21:35:23 /assateague/milky_way/raw/DSC03266.ARW
 1600.0    30.0    2.8    150.0 2019:07:29 21:36:24 /assateague/milky_way/raw/DSC03267.ARW
 3200.0    30.0    2.8    300.0 2019:07:29 21:37:15 /assateague/milky_way/raw/DSC03268.ARW
 6400.0    30.0    2.8    600.0 2019:07:29 21:38:03 /assateague/milky_way/raw/DSC03269.ARW
 6400.0    30.0    2.8    600.0 2019:07:29 21:39:21 /assateague/milky_way/raw/DSC03270.ARW
12800.0    30.0    2.8   1200.0 2019:07:29 21:40:30 /assateague/milky_way/raw/DSC03271.ARW
 6400.0    30.0    2.8    600.0 2019:07:29 21:42:38 /assateague/milky_way/raw/DSC03272.ARW

In [7]:
basepath = os.path.join(DATA,'astrophotography_data/MilkyWayPrettyBoy/')
dirpath = os.path.join(basepath,'800/light')
t = get_metadata(dirpath)
t.pprint_all()
dirpath = os.path.join(basepath,'1600/light')
t = get_metadata(dirpath)
t.pprint_all()
dirpath = os.path.join(basepath,'3200/light')
t = get_metadata(dirpath)
t.pprint_all()
dirpath = os.path.join(basepath,'6400/light')
t = get_metadata(dirpath)
t.pprint_all()
dirpath = os.path.join(basepath,'12800/light')
t = get_metadata(dirpath)
t.pprint_all()

 ISO  Exptime f/stop Exposure         Date                           File                  
----- ------- ------ -------- ------------------- -----------------------------------------
800.0     3.2    2.8      8.0 2019:11:01 20:50:38 /MilkyWayPrettyBoy/800/light/DSC03530.ARW
800.0     3.2    2.8      8.0 2019:11:01 20:50:44 /MilkyWayPrettyBoy/800/light/DSC03531.ARW
800.0     3.2    2.8      8.0 2019:11:01 20:50:49 /MilkyWayPrettyBoy/800/light/DSC03532.ARW
800.0     3.2    2.8      8.0 2019:11:01 20:50:54 /MilkyWayPrettyBoy/800/light/DSC03533.ARW
800.0     3.2    2.8      8.0 2019:11:01 20:51:00 /MilkyWayPrettyBoy/800/light/DSC03534.ARW
800.0     3.2    2.8      8.0 2019:11:01 20:51:05 /MilkyWayPrettyBoy/800/light/DSC03535.ARW
800.0     3.2    2.8      8.0 2019:11:01 20:51:10 /MilkyWayPrettyBoy/800/light/DSC03536.ARW
800.0     3.2    2.8      8.0 2019:11:01 20:51:15 /MilkyWayPrettyBoy/800/light/DSC03537.ARW
800.0     3.2    2.8      8.0 2019:11:01 20:51:20 /MilkyWayPrettyBoy/800/light/D

 ISO   Exptime f/stop Exposure         Date                           File                   
------ ------- ------ -------- ------------------- ------------------------------------------
3200.0     3.2    2.8     32.0 2019:11:01 21:02:35 /MilkyWayPrettyBoy/3200/light/DSC03650.ARW
3200.0     3.2    2.8     32.0 2019:11:01 21:02:42 /MilkyWayPrettyBoy/3200/light/DSC03651.ARW
3200.0     3.2    2.8     32.0 2019:11:01 21:02:47 /MilkyWayPrettyBoy/3200/light/DSC03652.ARW
3200.0     3.2    2.8     32.0 2019:11:01 21:02:52 /MilkyWayPrettyBoy/3200/light/DSC03653.ARW
3200.0     3.2    2.8     32.0 2019:11:01 21:02:57 /MilkyWayPrettyBoy/3200/light/DSC03654.ARW
3200.0     3.2    2.8     32.0 2019:11:01 21:03:03 /MilkyWayPrettyBoy/3200/light/DSC03655.ARW
3200.0     3.2    2.8     32.0 2019:11:01 21:03:08 /MilkyWayPrettyBoy/3200/light/DSC03656.ARW
3200.0     3.2    2.8     32.0 2019:11:01 21:03:13 /MilkyWayPrettyBoy/3200/light/DSC03657.ARW
3200.0     3.2    2.8     32.0 2019:11:01 21:03:18 /MilkyWay

  ISO   Exptime f/stop Exposure         Date                            File                   
------- ------- ------ -------- ------------------- -------------------------------------------
12800.0     3.2    2.8    128.0 2019:11:01 21:14:25 /MilkyWayPrettyBoy/12800/light/DSC03770.ARW
12800.0     3.2    2.8    128.0 2019:11:01 21:14:31 /MilkyWayPrettyBoy/12800/light/DSC03771.ARW
12800.0     3.2    2.8    128.0 2019:11:01 21:14:36 /MilkyWayPrettyBoy/12800/light/DSC03772.ARW
12800.0     3.2    2.8    128.0 2019:11:01 21:14:42 /MilkyWayPrettyBoy/12800/light/DSC03773.ARW
12800.0     3.2    2.8    128.0 2019:11:01 21:14:47 /MilkyWayPrettyBoy/12800/light/DSC03774.ARW
12800.0     3.2    2.8    128.0 2019:11:01 21:14:52 /MilkyWayPrettyBoy/12800/light/DSC03775.ARW
12800.0     3.2    2.8    128.0 2019:11:01 21:14:58 /MilkyWayPrettyBoy/12800/light/DSC03776.ARW
12800.0     3.2    2.8    128.0 2019:11:01 21:15:03 /MilkyWayPrettyBoy/12800/light/DSC03777.ARW
12800.0     3.2    2.8    128.0 2019:11: